# Code

## Установка библиотек и функций

In [ ]:
#@title Установка и импорт библиотек
!pip  install  tiktoken==0.4.0  langchain==0.0.231 openai==0.27.8 faiss-cpu==1.7.4 gspread oauth2client nltk pydantic==1.10.8 openpyxl

In [ ]:
import gdown
from langchain.llms import OpenAI
from langchain.docstore.document import Document
import requests
#database
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.text_splitter import CharacterTextSplitter
import re
from langchain.document_loaders import TextLoader
import ipywidgets as widgets
import gspread
import re
import getpass
import os
import openai
import tiktoken
from io import BytesIO
import openpyxl
import pandas as pd
from tqdm.auto import tqdm
import IPython

In [ ]:
MODEL_GPT_3_5_TURBO_16K = ['gpt-3.5-turbo-1106', 0.001, 0.002]  # 16K context window, [1], [2] - price per 1k tokens
MODEL_GPT_3_5_TURBO = ['gpt-3.5-turbo-0613', 0.0015, 0.002]  # 4,096 tokens, Shutdown date 2024-06-13
MODEL_GPT_3_5_TURBO_INSTRUCT = ['gpt-3.5-turbo-instruct', 0.0015, 0.002]  # 4,096 tokens
MODEL_GPT_4 = ['gpt-4', 0.03, 0.06]  # 8,192 tokens
MODEL_GPT_4_TURBO = ['gpt-4-1106-preview', 0.01, 0.03]  # 128k tokens	content
SELECT_MODEL_GPT = MODEL_GPT_3_5_TURBO_16K # выбери нужную модель

In [ ]:
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

Для загрузки переменных окружения также можно использовать python-dotenv. Просто создайте файл .env и определите в нем переменную OPENAI_API_KEY = "ваш ключ OPENAI_API_KEY".

In [ ]:
# load_dotenv()

# # API configuration
# openai.api_key = os.getenv("OPENAI_API_KEY")

# # for LangChain
# os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
# os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

In [ ]:
#@title функция подсчета токенов в строке
def num_tokens_from_string(string: str, encoding_name: str) -> int:
      """Returns the number of tokens in a text string."""
      encoding = tiktoken.get_encoding(encoding_name)
      num_tokens = len(encoding.encode(string))
      return num_tokens

In [ ]:
#@title функция для загрузки документа по ссылке из гугл драйв
def load_document_text(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

In [ ]:
#@title  функция для загрузки txt файла по ссылке
def load_text(url: str) -> str:
    # Download the document as plain text
    response = requests.get(url)
    response.raise_for_status()
    text = response.text

    return text

In [ ]:
#@title  Функция деления БЗ на чанки с помощью MarkdownHeaderTextSplitter
def MD_split_documents(file_path: str) -> str:
    # Чтение текстового файла
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    headers_to_split_on = [
        ("#", "Header1"),
        ("##", "Header2"),
        ("###", "Header3"),
        ("####", "Header4"),
    ]

    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    documents = markdown_splitter.split_text(text)

    return documents

In [ ]:
# @title функция для загрузки таблицы по ссылке из гугл драйв
#def load_xls_pd(url: str) -> str:
def load_xls_pd(url: str, sheet_name: str = 0) -> str:

    # Extract the document ID from the URL
    match_ = re.search('/spreadsheets/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Sheets URL')
    doc_id = match_.group(1)

    if sheet_name is None:
        raise ValueError('Invalid sheet_name')

    # Download the table as pandas
    response = requests.get(f'https://docs.google.com/spreadsheets/d/{doc_id}/export?format=xlsx')
    response.raise_for_status()     #проверяет статус код ответа. Если получен ответ с кодом ошибки (4xx или 5xx), вызывается исключение HTTPError.
    #data = pd.read_excel(BytesIO(response.content), sheet_name=0)  #чтение первого листа из книги
    data = pd.read_excel(BytesIO(response.content),sheet_name = sheet_name, header=None) #чтение конкретного листа из Книги Excel

    return data

### Функция запроса к ChatGPT

In [ ]:
def answer_index(doc, temp=1.0):

        input_text = f'{doc.page_content}'

        messages = [
            {"role": "system", "content": prompt},
            {"role": "user", "content": f"Часть базы знаний для составления вопросов:\n{input_text}"}
            ]

        completion = openai.ChatCompletion.create(
            model=SELECT_MODEL_GPT[0],
            messages=messages,
            temperature=temp
            )
        # подсчет стоимости запроса, можно раскоментировать
        # print(f'{completion["usage"]["total_tokens"]} токенов использовано всего (вопрос-ответ).')
        # print('ЦЕНА запроса с ответом :', (SELECT_MODEL_GPT[1])*(completion["usage"]["prompt_tokens"]/1000) + (SELECT_MODEL_GPT[2])*(completion["usage"]["completion_tokens"]/1000), ' $')
        # print('===========================================: \n')

        answer = completion.choices[0].message.content

        # import IPython
        # IPython.display.Markdown(answer)

        return  answer

In [ ]:
# @title Загрузка таблицы для тестирования
# Шаг 1: Загрузить xlsx файл из Google Drive
google_sheet_url = "https://docs.google.com/spreadsheets/d/1U4J6GgcamnRmV7mjhcbXxhTIsJwOVyKPqAVnimcHtR0/edit?usp=sharing"
sheet_name='Вопросы для тестирования'     # вставить название листа для заполнения вопросами

#data = load_xls_pd(google_sheet_url)   #если в таблице только один лист
df = load_xls_pd(google_sheet_url, sheet_name)

In [ ]:
# проверка
df.head()

In [ ]:
# установить первую строку в виде строки заголовков столбцов
df.set_axis(df.iloc[0], axis='columns', inplace=True)
# проверка
df.info()
df.head()

In [ ]:
#df['Чанки'] = pd.Series(dtype=str)   # так можно добавить столбец "Чанки", если его нет

In [ ]:
#df = df.drop('Чанки', axis=1)         # а так можно удалить столбец

In [ ]:
#df.insert(loc=10, column='Чанки', value='') #2ой способ добавить пустой столбец "Чанки", если он отсутствует 11 столбцом

### Здесь можно загрузить или написать промпт

In [ ]:
# prompt = load_text('https://raw.githubusercontent.com/terrainternship/GPT_Saratov_med_g/main/Minkin_Alex/prompt_for_question.txt')

In [ ]:
# IPython.display.Markdown(prompt)

In [ ]:
prompt = '''
Есть следующее описание проекта:
Цель проекта: Разработать информационного нейропомощника для паллиативных пациентов, их родственников и лиц, осуществляющих непосредственных уход.
Описание проблемы: Оказание паллиативной медицинской помощи многокомпонентный процесс, включающий в себя непосредственные медицинские манипуляции, но психологическую работу врач-пациент. В настоящее время складывается ситуация, когда паллиативные пациенты и их родственники или другие лица, осуществляющие уход (опекун) остаются один на один с проблемой бремени заболевания без возможности доступа и верной информации. В частности не имеют четкого представления о перспективах заболевания, плане последовательности при маршрутизации пациентов с подобной проблемой, не знают о возможностях социальной поддержки государства и условиях ее получения и тд.
Помоги мне придумать вопросы к части базы знаний, предоставленной ниже, для тестирования алгоритма работы и правильности составления базы знаний.
Нужно придумать 3 вопроса. Вопросы должны быть максимально похожи на вопросы, заданные человеком о помощи паллиативным пациентам.
Вопросы нумеровать не нужно. Кроме вопросов больше ничего предоставлять не нужно.
'''

### Для выполнения запросов вручную

In [ ]:
# def run_dialog():

#     while True:
#         user_question = input('\nКлиент: ')
#         if ((user_question.lower() == 'stop') or (user_question.lower() == 'стоп')):
#             break
#         answer = answer_index(user_question)
#         print('\nМенеджер: ', answer + '\n\n')

#     return


In [ ]:
# закоментируй для сохранения ответов и чанков в таблицу
# verbose=1
# run_dialog()

## Сохранение ответов ChatGPT в таблицу

### Предварительно нужно загрузить файл базы знаний 'database.md' в колаб

In [ ]:
#Если заново собирать векторную базу из базы знаний, то нужно раскоментировать этот код
'''
!git clone -b develop https://github.com/proekt-GPT.git
!cp /content/proekt-GPT/knowledge/database.md /content/database.md
!rm -r /content/proekt-GPT
knowledge_base = load_file_knowledge('database.md')
'''

In [ ]:
# загрузка БЗ с GitHub
knowledgebase = load_text('https://raw.githubusercontent.com/terrainternship/GPT_Saratov_med_g/main/structured_data_base_081223_rtf.txt')

# loader = TextLoader('https://raw.githubusercontent.com/terrainternship/GPT_Saratov_med_g/main/structured_data_base_081223_rtf.txt')

# documents = loader.load()

In [ ]:
knowledgebase

In [ ]:
docs = []
text_splitter = CharacterTextSplitter(separator="<chunk>", chunk_size=1000, chunk_overlap=0)
for chunk in text_splitter.split_text(knowledgebase):
    docs.append(Document(page_content=chunk, metadata={}))

In [ ]:
len(docs)     # количество чанков в базе

### можно проверить правильно ли сформированы чанки

In [ ]:
docs[1]

In [ ]:
docs[0].metadata

In [ ]:
docs[0].page_content

In [ ]:
type(docs[0])

In [ ]:
df = df.iloc[0:0]   # удаление всех строк в таблице

In [ ]:
# @title Запуск алгоритма создания вопросов и их запись в датафрейм
count = 0   # Переменная для подсчета количества строк, обработанных по условию
print('Количество обработанных запросов: ')
# Цикл для обхода чанков
for doc in tqdm(docs):
        # Ваш код для обработки данных текущей строки
        answers = answer_index(doc)
        answers = answers.split('\n') # Разделите строку на список строк по символу новой строки
        # Удалить последние две строки, т.к. там GPT пишет лишнюю обобщенную информацию
        # answers = answers[:-2]
        # # Отделяем первую и вторую строки для раздела и подраздела, остальное для вопросов
        # section, subsection, *questions = answers
        # # Создаем DataFrame для каждого вопроса по отдельности
        # frames = [pd.DataFrame({"Раздел": [section], "Подраздел": [subsection], "Вопрос": [question]})
        #           for question in questions]
        # Объединяем все DataFrame'ы в один
        # Создайте новый DataFrame и присоедините его к существующему
        new_rows = pd.DataFrame({"Вопрос": answers}, index = range(len(answers)))
        # Добавляем новые строки в исходный DataFrame
        df = pd.concat([df, new_rows], ignore_index=True)
        # Увеличиваем значение счетчика и проверяем, достигли ли мы нужного количества запросов
        count += 1
        if count >= 10:          # можно ограничить количество задаваемых вопросов к ChatGPT
            break

In [ ]:
# df[df['Тема'] == 'Аксесуары'][['Вопрос', 'Ответ GPT', 'Чанки']].head(15)   #проверка записей в таблице

In [ ]:
# df[(df['Тема'] == 'Аксесуары') & (df['Ответ GPT'].isna())].head(10) #проверка записей в таблице

In [ ]:
df    # просмотр таблицы

In [ ]:
#@title Загрузка df в таблицу 'questions.xlsx'
df.to_excel('questions.xlsx')

Далее:
- можно сохранить полученную таблицу на диск или скопировать в Гугл диск